In [41]:
import pandas as pd
import sqlite3
import re
from contextlib import closing
from datetime import datetime

In [42]:
db = "test.db"

query_sessions = '''
SELECT id, notes FROM session
'''

def get_uid(name:str)->str:
    return re.findall(r"\d+", name)

def get_date(name:int)->datetime:
    return datetime.strptime(
        re.findall(r"\d+", str(name))[0],
        r'%Y%m%d%H%M%S'
    ).strftime(r"%c")

def list_sessions():
    with closing(sqlite3.connect(db)) as conn:
        sessions = pd.read_sql(query_sessions, conn)
        sessions["date"] = sessions["id"].apply(get_date)

        return sessions

In [43]:
sessions = list_sessions()
sessions

,id,notes,date
0,20240719165621,pruebas,Fri Jul 19 16:56:21 2024
1,20240719165701,wawa,Fri Jul 19 16:57:01 2024
2,20240719170709,,Fri Jul 19 17:07:09 2024
3,20240719171008,isis con ximena,Fri Jul 19 17:10:08 2024
4,20240719171743,isis con ximena,Fri Jul 19 17:17:43 2024
5,20240719172230,isis con ximena,Fri Jul 19 17:22:30 2024
6,20240719172931,isis con ximena,Fri Jul 19 17:29:31 2024


In [46]:
def get_events(uid:int):
    with closing(sqlite3.connect(db)) as conn:
        return pd.read_sql(
            f"SELECT * FROM events_{uid}", conn
        )
    
events = get_events(20240719172931)
events.head()

,time,event
0,101,inicio
1,104,buena roja
2,106,mala roja
3,109,mala negra
4,111,mala negra


In [47]:
def get_session(uid:int, start:int, stop:int):
    with closing(sqlite3.connect(db)) as conn:
        return pd.read_sql(
            f"SELECT * FROM session_{uid} LIMIT {stop-start} OFFSET {start}", conn
        )
    
session = get_session(20240719172931, events.iloc[0, 0], events.iloc[-1,0])
session

,signal_strength0,attention0,meditation0,delta0,theta0,low_alpha0,high_alpha0,low_beta0,high_beta0,low_gamma0,...,attention4,meditation4,delta4,theta4,low_alpha4,high_alpha4,low_beta4,high_beta4,low_gamma4,high_gamma4
0,0,63,74,54536,30059,5059,7864,6498,6381,4507,...,50,56,540559,43523,37860,36263,22065,12273,14431,39218
1,0,63,66,1890553,151030,64078,11325,7827,18788,25194,...,40,30,297904,152433,7907,5662,11573,7721,5891,6159
2,0,74,60,676968,45288,3062,21632,28849,37413,16444,...,38,35,41727,72638,21741,22655,18451,22221,8039,20741
3,0,81,44,1687294,159248,6507,31231,17700,72989,21773,...,34,53,1066074,371146,53904,42032,38540,20739,19733,15735
4,0,70,41,111491,75140,3646,13916,7246,4346,3978,...,37,47,598886,50337,33786,12194,32903,12042,10675,6021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0,50,70,188103,44395,19209,27845,8115,14638,11413,...,44,64,61044,43196,47662,11797,10785,25940,10836,14449
132,0,50,57,379232,99099,8666,19972,6811,18049,12059,...,56,80,331951,42837,34465,12057,33716,11236,5514,7798
133,0,60,54,258916,128159,20747,16120,16725,17259,23007,...,50,66,37296,227257,11856,18913,8921,20864,13067,9371
134,0,63,64,62293,16611,10113,16971,2914,7045,1431,...,56,66,35838,40892,16321,16329,16239,12091,5398,10813
